In [ ]:
# Import Dependencies
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sqlalchemy import create_engine
from config import sql_pw

In [ ]:
# Loading mock dataset for Provisional Machine Learning Model
# movies_df = pd.read_csv("Potential_Datasets/datasets_2745_4700_movies.csv", encoding = "ISO-8859-1")
# movies_df.head()

In [ ]:
# Extracting Dataset from S3 Bucket
url = "https://utbootcamp-bucket.s3.amazonaws.com/<folder-name>/datasets_2745_4700_movies.csv"
movies_df = 
movies_df.head()

# Performing inital cleaning of data

In [ ]:
# Droping Identification column (will need to apply NLP to Description)
movies_df_clean = movies_df.copy()
movies_df_clean = movies_df_clean.drop(["name"], axis=1)
movies_df_clean.head()

In [ ]:
# Generate our categorical variable list
movies_cat = movies_df_clean.dtypes[movies_df_clean.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
movies_df_clean[movies_cat].nunique()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(movies_df_clean[movies_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(movies_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
movies_df_clean = movies_df_clean.merge(encode_df,left_index=True, right_index=True)
movies_df_clean = movies_df_clean.drop(movies_cat,1)
movies_df_clean.head()

In [ ]:
# Converting score to recomendation value
movies_df_clean.loc[movies_df_clean.score >= 7.0, "recomendation"] = 2
movies_df_clean.loc[(movies_df_clean.score < 7.0) & (movies_df_clean.score > 5.0), "recomendation"] = 1
movies_df_clean.loc[movies_df_clean.score <= 5.0, "recomendation"] = 0
movies_df_clean.head()

# Load Dataset into PostgreSQL Database

In [ ]:
try:
    # Create Database Engine and Load Dataset into Database
    connection_string = "postgres://database-1.cuz2axsi4p0x.us-east-2.rds.amazonaws.com/<database-name>"
    engine = create_engine(connection_string)
    movies_df_clean.to_sql(name="<table-name>", con=engine, if_exists="replace")
    
    print("Loaded Movies Data into SQL")
except:
    print("Failed to load Movies Data into SQL")